# Discrete Convolution and Correlation

This notebook illustrates the (discrete) linear and circular convolutions.

* Graham, L., D. E. Knuth, and O. Patashnik (1994). Concrete mathematics: a foundation for computer science, 2 ed.: Addison-Wesley Publishing Company. ISBN 0-201-55802-5

* Yilmaz, Öz (2001). Seismic Data Analysis: Processing, Inversion, and Interpretation of Seismic Data, 2 ed.: Society of Exploration Geophysicists. ISBN 1-56080-098-4

* Oppenheim, A. V., and Schafer, R., W. (2010). Discrete-Time Signal Processing, 3 ed.: Pearson. ISBN 0-13-198842-5

In [1]:
import numpy as np
from scipy.fft import fft, ifft
from scipy.linalg import toeplitz, circulant, dft
import matplotlib.pyplot as plt
import my_functions as mfun

## Linear convolution

Let $\mathbf{a}$ be an $N_{a} \times 1$ vector and $\mathbf{b}$ be an $N_{b} \times 1$ vector. The *linear convolution* of $\mathbf{a}$ and $\mathbf{b}$ generates a $N_{w} \times 1$ vector $\mathbf{w}$, where $N_{w} = N_{a} + N_{b} - 1$, whose $i$th element is defined as follows (Oppenheim and Shafer, 2010, p. 661):

$$
w_{i} = \sum\limits_{j = -\infty}^{\infty} b_{i - j} \, a_{j} \: .
$$

The linear convolution is schematically represented as follows (e.g., Yilmaz, 2001, p. 39):

In [2]:
mfun.linear_convolution_scheme(Na=4, Nb=3)

Linear convolution:
w_0 = (b_0 * a_0) + (  0 * a_1) + (  0 * a_2) + (  0 * a_3) + (  0 *  0) + (  0 *  0) + (  0 *  0) + (  0 *  0)
w_1 = (b_1 * a_0) + (b_0 * a_1) + (  0 * a_2) + (  0 * a_3) + (  0 *  0) + (  0 *  0) + (  0 *  0) + (  0 *  0)
w_2 = (b_2 * a_0) + (b_1 * a_1) + (b_0 * a_2) + (  0 * a_3) + (  0 *  0) + (  0 *  0) + (  0 *  0) + (  0 *  0)
w_3 = (  0 * a_0) + (b_2 * a_1) + (b_1 * a_2) + (b_0 * a_3) + (  0 *  0) + (  0 *  0) + (  0 *  0) + (  0 *  0)
w_4 = (  0 * a_0) + (  0 * a_1) + (b_2 * a_2) + (b_1 * a_3) + (b_0 *  0) + (  0 *  0) + (  0 *  0) + (  0 *  0)
w_5 = (  0 * a_0) + (  0 * a_1) + (  0 * a_2) + (b_2 * a_3) + (b_1 *  0) + (b_0 *  0) + (  0 *  0) + (  0 *  0)
  0 = (  0 * a_0) + (  0 * a_1) + (  0 * a_2) + (  0 * a_3) + (b_2 *  0) + (b_1 *  0) + (b_0 *  0) + (  0 *  0)


Toeplitz system:
 w_0 = |  b_0    0    0    0    0    0    0 |  a_0
 w_1 = |  b_1  b_0    0    0    0    0    0 |  a_1
 w_2 = |  b_2  b_1  b_0    0    0    0    0 |  a_2
 w_3 = |    0  b_2  b_1 

The scheme above shows that the convolution is a matrix-vector product:

$$
\begin{bmatrix}
\mathbf{w} \\
\hline
0
\end{bmatrix} = \mathbf{B} \, 
\begin{bmatrix}
\mathbf{a} \\
\hline
\mathbf{0}_{N_{b}}
\end{bmatrix} \quad ,
$$

where $\mathbf{B}$ is a Toeplitz matrix. Matrix-vetor products with Toeplitz matrices have some special properties (see the notebook `Toeplitz-circulant-matrix-vector.ipynb`) and can be computed by using the Fourier transform. Specifically, the linear convolution can be computed as follows:

$$
\begin{bmatrix}
\mathbf{w} \\
\hline
0
\end{bmatrix} 
= \mathbf{F}_{(N)}^{\ast} \Bigg\{ \left( \sqrt{N} \:\: \mathbf{F}_{(N)} 
\begin{bmatrix}
\mathbf{b} \\
\hline
\mathbf{0}_{N_{a}}
\end{bmatrix} \right) \circ 
\left( \mathbf{F}_{(N)} 
\begin{bmatrix}
\mathbf{a} \\
\hline
\mathbf{0}_{N_{b}}
\end{bmatrix} \right) \Bigg\}
$$

In [3]:
# number of data points in a
Na = 100

# data vector a
a = 10*np.random.rand(Na)

# number of data points in b
Nb = 80

# data vector b
b = 10*np.random.rand(Nb)

In [4]:
# number of elements in w
Nw = Na + Nb - 1

In [5]:
N = Na+Nb

In [6]:
# vector a padded with zeros
a_padd = np.hstack([a, np.zeros(Nb)])

In [7]:
# vector b padded with zeros
b_padd = np.hstack([b, np.zeros(Na)])

In [8]:
# Toeplitz matrix B
B = toeplitz(b_padd, np.zeros(N))

In [9]:
# linear convolution computed as a matrix-vector product
w_matvec = np.dot(B, a_padd)[:-1]

In [10]:
# linear convolution computed by FFT
DFT_a_padd = fft(x=a_padd, norm='ortho')
DFT_b_padd = fft(x=b_padd, norm='ortho')
w_fft = ifft(x=np.sqrt(N)*DFT_a_padd*DFT_b_padd, norm='ortho').real[:-1]

In [11]:
np.allclose(w_matvec, w_fft)

True

In [12]:
# linear convolution computed by using numpy.convolve
w_convolve = np.convolve(a, b, mode='full')

In [13]:
np.allclose(w_matvec, w_convolve)

True

## Circular convolution

Let $\mathbf{a}$ and $\mathbf{b}$ be $N \times 1$ vectors. The circular convolution of $\mathbf{a}$ and $\mathbf{b}$ generates an $N \times 1$ vector $\mathbf{w}$ whose $i$th element is defined as follows:

$$
w_{i} = \sum\limits_{j = 0}^{N-1} b_{(i - j)\text{mod}N} \, a_{j} \: .
$$

The **mod** function $x \, \text{mod} \, y$ (Graham et al., 1994, p. 82) computes the remainder of division of `x` by `y`. It can be rewritten as follows:

$$
x \, \text{mod} \, y = x - y \, \Big\lfloor \frac{x}{y} \Big\rfloor \: ,
$$

where $\lfloor \cdot \rfloor$ denotes the **floor** function (Graham et al., 1994, p. 67), which computes the
greatest integer less than or equal to its argument. The mod function is implemented in the routine [`numpy.mod`](https://numpy.org/doc/stable/reference/generated/numpy.mod.html).

In [14]:
mfun.circular_convolution_scheme(N=4)

Circular convolution:
w_0 = (b_0 * a_0) + (b_3 * a_1) + (b_2 * a_2) + (b_1 * a_3)
w_1 = (b_1 * a_0) + (b_0 * a_1) + (b_3 * a_2) + (b_2 * a_3)
w_2 = (b_2 * a_0) + (b_1 * a_1) + (b_0 * a_2) + (b_3 * a_3)
w_3 = (b_3 * a_0) + (b_2 * a_1) + (b_1 * a_2) + (b_0 * a_3)


Circulant system:
 w_0 = |  b_0  b_3  b_2  b_1 |  a_0
 w_1 = |  b_1  b_0  b_3  b_2 |  a_1
 w_2 = |  b_2  b_1  b_0  b_3 |  a_2
 w_3 = |  b_3  b_2  b_1  b_0 |  a_3


PAREI AQUI

Notice that the system above satisfies the necessary conditions (see the notebook `Toeplitz-circulant-matrix-vector.ipynb`) so that the circular convolution can be computed by using the Fourier transform. Specifically, the circular convolution can be computed as follows:

$$
\mathbf{w} = \mathbf{F}_{(N)}^{\ast} \Bigg\{ \left( \sqrt{N} \:\: 
\mathbf{F}_{(N)} \, \mathbf{b} \right) \circ 
\left( \mathbf{F}_{(N)} \, \mathbf{a} \right) \Bigg\}
$$

In [15]:
# number of data points
N = 100

# data vector a
a = 10*np.random.rand(N)

# data vector b
b = 10*np.random.rand(N)

In [16]:
# Circulant matrix C
C = circulant(b)

In [17]:
# circular convolution computed as a matrix-vector product
w_matvec = np.dot(C, a)

In [18]:
# circular convolution computed by FFT
DFT_a = fft(x=a, norm='ortho')
DFT_b = fft(x=b, norm='ortho')
w_fft = ifft(x=np.sqrt(N)*DFT_a*DFT_b, norm='ortho').real

In [19]:
np.allclose(w_matvec, w_fft)

True

## Comparison between linear and circular convolutions

In [ ]:
mfun.linear_convolution_scheme(Na=3, Nb=3)

In [ ]:
mfun.circular_convolution_scheme(N=3)

In [ ]:
mfun.circular_convolution_scheme(N=6)

Circular convolution generates the same result as linear convolution if the input vectors are padded with zeros.

## Crosscorrelation and Autocorrelation

Let $\mathbf{a}$ be an $N_{a} \times 1$ vector and $\mathbf{b}$ be an $N_{b} \times 1$ vector. The *crosscorrelation* of $\mathbf{a}$ and $\mathbf{b}$ generates a $N \times 1$ vector $\mathbf{w}$, where $N = N_{a} + N_{b} - 1$, whose $i$th element is defined as follows:

$$
w_{i} = \sum\limits_{j = 0}^{N-1} b_{i + j} \, a_{j} \: .
$$

The crosscorrelation is schematically represented as follows (e.g., Yilmaz, 2001, p. 40):

In [ ]:
mfun.crosscorrelation_scheme(Na=4, Nb=3)

In [ ]:
# number of data points in a
Na = 100

# data vector a
a = 10*np.random.rand(Na)

# number of data points in b
Nb = 80

# data vector b
b = 10*np.random.rand(Nb)

In [ ]:
# number of elements in w
Nw = Na + Nb - 1

In [ ]:
N = Na + Nb

In [ ]:
# vector a padded with zeros
a_padd = np.hstack([a, np.zeros(Nb)])

In [ ]:
# vector b padded with zeros
b_padd = np.hstack([b[::-1], np.zeros(Na)])

In [ ]:
# Toeplitz matrix B
B = toeplitz(b_padd, np.zeros(N))

In [ ]:
# crosscorrelation computed as a matrix-vector product
w_matvec = np.dot(B, a_padd)[:-1]

In [ ]:
# crosscorrelation computed by FFT
DFT_a_padd = fft(x=a_padd, norm='ortho')
DFT_b_padd = fft(x=b_padd, norm='ortho')
w_fft = ifft(x=np.sqrt(N)*DFT_a_padd*DFT_b_padd, norm='ortho').real[:-1]

In [ ]:
np.allclose(w_matvec, w_fft)

In [ ]:
# crosscorrelation computed by using numpy.convolve
w_correlate = np.correlate(a, b, mode='full')

In [ ]:
np.allclose(w_matvec, w_correlate)

If vectors $\mathbf{a}$ and $\mathbf{b}$ are the same, the crosscorrelation is called *autocorrelation*.

In [ ]:
mfun.autocorrelation_scheme(N=3)